In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/clean_raw_data.csv
/kaggle/input/clean_data_mean.csv
/kaggle/input/raw_data.csv
/kaggle/input/clean_data_GCN.csv
/kaggle/input/FillZero_minmax_baseline/clean_data_minmax_fill-zero.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/val/val_week1_2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/test/test_week2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/clean_data_week2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_3.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_4.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_1.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_5.csv
/kaggle/input/FillZero_minmax_baseline/clean_week3/val/val_week1_2_3.csv
/kaggle/input/FillZero_minmax_baseline/clean_week3/test/test_week3.csv
/kaggle/input/FillZero_minmax_bas

In [2]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from kerastuner.tuners import RandomSearch
from sklearn.model_selection import StratifiedKFold
import time
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_recall_fscore_support, roc_auc_score

<ipython-input-2-63192117ceec>:7: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [3]:
# Biến global cho base path
BASE_PATH = "/kaggle/input/FillZero_minmax_baseline"
# Tuần và số phần fold
weeks = ['week1', 'week2', 'week3', 'week4']
fold_parts = 5

# Tạo five_fold_files
five_fold_files = {
    week: [
        f"{BASE_PATH}/clean_{week}/train/5-folds/data_part_{i}.csv"
        for i in range(1, fold_parts + 1)
    ]
    for week in weeks
}

# Tạo file_validation
file_validation = {
    'week1': [f"{BASE_PATH}/clean_week1/val/val_week1.csv"],
    'week2': [f"{BASE_PATH}/clean_week2/val/val_week1_2.csv"],
    'week3': [f"{BASE_PATH}/clean_week3/val/val_week1_2_3.csv"],
    'week4': [f"{BASE_PATH}/clean_week4/val/val_week1_2_3_4.csv"]
}

# Tạo file_test
file_test = {
    week: [f"{BASE_PATH}/clean_{week}/test/test_{week}.csv"]
    for week in weeks
}


In [4]:
# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    return focal_loss_fixed

# Tạo hàm train cho từng tuần
def train_week_model(week_number, file_paths_train, file_validataion):
   # Đọc dữ liệu
    train_data = pd.read_csv(file_paths_train)
    val_data = pd.read_csv(file_validataion)
    
    # Tách đặc trưng và nhãn
    X_train = train_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_train = train_data["classification_encoded"]

    X_val = val_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_val = val_data["classification_encoded"]
    

    # Áp dụng Over-sampling cho dữ liệu huấn luyện bằng SMOTE
    oversampler = SMOTE(sampling_strategy='auto', random_state=42)
    X_train_res, y_train_res = oversampler.fit_resample(X_train, y_train)

    # One-hot encode nhãn
    y_train_res = tf.keras.utils.to_categorical(y_train_res, num_classes=5)
    y_val = tf.keras.utils.to_categorical(y_val, num_classes=5)

    # Xây dựng mô hình 4-layer stacked LSTM
    def build_model(hp):
        model = tf.keras.Sequential()
        
        # LSTM layer 1
        model.add(layers.LSTM(
            units=hp.Int('units_1', min_value=32, max_value=256, step=32),
            return_sequences=True,
            input_shape=(X_train_res.shape[1], 1)
        ))
        model.add(layers.Dropout(rate=hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)))
        
        # LSTM layer 2
        model.add(layers.LSTM(
            units=hp.Int('units_2', min_value=32, max_value=256, step=32),
            return_sequences=True
        ))
        model.add(layers.Dropout(rate=hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)))
        
        # LSTM layer 3
        model.add(layers.LSTM(
            units=hp.Int('units_3', min_value=32, max_value=256, step=32),
            return_sequences=True
        ))
        model.add(layers.Dropout(rate=hp.Float('dropout_3', min_value=0.1, max_value=0.5, step=0.1)))
        
        # LSTM layer 4
        model.add(layers.LSTM(
            units=hp.Int('units_4', min_value=32, max_value=256, step=32),
            return_sequences=False
        ))
        model.add(layers.Dropout(rate=hp.Float('dropout_4', min_value=0.1, max_value=0.5, step=0.1)))

        # Lớp đầu ra
        model.add(layers.Dense(5, activation='softmax'))

        # Compile với Focal Loss
        model.compile(optimizer=tf.keras.optimizers.Adam(
                          learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                      loss=focal_loss(gamma=2., alpha=0.25),
                      metrics=['accuracy'])
        return model

    # Khởi tạo RandomSearch tuner
    tuner = RandomSearch(
        build_model,
        objective='val_accuracy',
        max_trials=10,
        executions_per_trial=1,
        directory='my_dir',
        project_name=f'lstm_tuning_week{week_number}'
    )

    # Tìm kiếm siêu tham số tốt nhất
    tuner.search(X_train_res, y_train_res,
                 epochs=20,
                 validation_data=(X_val, y_val),
                 batch_size=32)

    # Trả về kết quả tối ưu cho tuần
    best_params = tuner.get_best_hyperparameters(num_trials=1)[0]
    return best_params


In [5]:
# Định nghĩa đường dẫn đến dữ liệu cho từng tuần
file_paths_train = {
    week: f"{BASE_PATH}/clean_{week}/train/clean_data_{week}.csv"
    for week in weeks
}

# Định nghĩa file_validation theo quy luật riêng
file_validation = {
    f"week{idx + 1}": f"{BASE_PATH}/clean_week{idx + 1}/val/val_week{'_'.join(str(i) for i in range(1, idx + 2))}.csv"
    for idx in range(len(weeks))
}

In [6]:
# Tìm tham số tốt nhất cho từng tuần
best_params_week1 = train_week_model(1, file_paths_train["week1"], file_validation["week1"])
best_params_week2 = train_week_model(2, file_paths_train["week2"], file_validation["week2"])
best_params_week3 = train_week_model(3, file_paths_train["week3"], file_validation["week3"])
best_params_week4 = train_week_model(4, file_paths_train["week4"], file_validation["week4"])

# In thông tin chi tiết các tham số tối ưu
print("Best Parameters for Week 1:")
for param_name in best_params_week1.values.keys():
    print(f"{param_name}: {best_params_week1.get(param_name)}")

print("\nBest Parameters for Week 2:")
for param_name in best_params_week2.values.keys():
    print(f"{param_name}: {best_params_week2.get(param_name)}")

print("\nBest Parameters for Week 3:")
for param_name in best_params_week3.values.keys():
    print(f"{param_name}: {best_params_week3.get(param_name)}")

print("\nBest Parameters for Week 4:")
for param_name in best_params_week4.values.keys():
    print(f"{param_name}: {best_params_week4.get(param_name)}")


Trial 10 Complete [00h 04m 16s]
val_accuracy: 0.7785235047340393

Best val_accuracy So Far: 0.7815741300582886
Total elapsed time: 00h 44m 17s
Best Parameters for Week 1:
units_1: 192
dropout_1: 0.2
units_2: 96
dropout_2: 0.1
units_3: 64
dropout_3: 0.4
units_4: 192
dropout_4: 0.5
learning_rate: 0.002143981088964242

Best Parameters for Week 2:
units_1: 192
dropout_1: 0.30000000000000004
units_2: 64
dropout_2: 0.2
units_3: 256
dropout_3: 0.4
units_4: 224
dropout_4: 0.2
learning_rate: 0.002028760732752755

Best Parameters for Week 3:
units_1: 96
dropout_1: 0.30000000000000004
units_2: 64
dropout_2: 0.30000000000000004
units_3: 64
dropout_3: 0.5
units_4: 256
dropout_4: 0.5
learning_rate: 0.0026385629878377535

Best Parameters for Week 4:
units_1: 64
dropout_1: 0.2
units_2: 192
dropout_2: 0.2
units_3: 96
dropout_3: 0.5
units_4: 224
dropout_4: 0.2
learning_rate: 0.0001781435007513765


In [7]:
# Danh sách tham số tốt nhất
best_params = {
    "week1": best_params_week1,
    "week2": best_params_week2,
    "week3": best_params_week3,
    "week4": best_params_week4
}

In [8]:
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras import backend as K

# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    
    return focal_loss_fixed

# Xây dựng mô hình
def build_lstm_model(params):
    model = tf.keras.Sequential()
    
    # LSTM layer 1
    model.add(layers.LSTM(
        units=params.get('units_1', 32),
        return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]))
    )
    model.add(layers.Dropout(rate=params.get('dropout_1', 0.2)))

    # LSTM layer 2
    model.add(layers.LSTM(
        units=params.get('units_2', 32),
        return_sequences=True
    ))
    model.add(layers.Dropout(rate=params.get('dropout_2', 0.2)))

    # LSTM layer 3
    model.add(layers.LSTM(
        units=params.get('units_3', 32),
        return_sequences=True
    ))
    model.add(layers.Dropout(rate=params.get('dropout_3', 0.2)))

    # LSTM layer 4
    model.add(layers.LSTM(
        units=params.get('units_4', 32),
        return_sequences=False
    ))
    model.add(layers.Dropout(rate=params.get('dropout_4', 0.2)))

    # Output layer
    model.add(layers.Dense(5, activation='softmax'))  # 5 classes in output

    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=params['learning_rate']),
        loss=focal_loss(gamma=params.get('gamma', 2.), alpha=params.get('alpha', 0.25)),
        metrics=['accuracy']
    )
    
    return model


In [9]:
# Biến lưu kết quả tổng quát
overall_results_5folds = []

# Lặp qua từng tuần
for week, file_paths in five_fold_files.items():
    print(f"\nProcessing {week} with best parameters...")
    params = best_params[week].values
    print(f"best parameters for {week}: {params}")
    
    # Biến lưu kết quả cho từng tuần
    week_results = {
        "week": week,
        "accuracy_per_fold": [],
        "precision_per_label": [],
        "recall_per_label": [],
        "f1_score_per_label": [],
        "auc_roc_per_label": [],    # AUC từng lớp
        "auc_roc_macro": [],        # AUC macro
        "auc_roc_weighted": [],     # AUC weighted (tự tính)
        "precision_macro": [],
        "recall_macro": [],
        "f1_macro": [],
        "precision_weighted": [],
        "recall_weighted": [],
        "f1_weighted": [],
        "confusion_matrices": [],
        "train_times": [],
        "test_times": []
    }

    # Lặp qua từng fold
    for i in range(len(file_paths)):
        print(f"Fold {i+1}: Using file {file_paths[i]} as test set")
        
        # Tải dữ liệu
        test_data = pd.read_csv(file_paths[i])
        train_data = pd.concat([pd.read_csv(file_paths[j]) for j in range(len(file_paths)) if j != i])
        
        # Tách X và y
        X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                           "course_id", "school", "enroll_time", "classification"])
        y_train = to_categorical(train_data['classification_encoded'], num_classes=5)
        
        X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                         "course_id", "school", "enroll_time", "classification"])
        y_test = to_categorical(test_data['classification_encoded'], num_classes=5)

        # Reshape dữ liệu cho LSTM
        X_train = X_train.to_numpy().reshape((X_train.shape[0], 1, X_train.shape[1]))
        X_test = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))

        # Xây dựng mô hình với tham số tốt nhất
        model = build_lstm_model(params)
        
        # Bắt đầu tính thời gian huấn luyện
        start_train = time.time()
        model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=32)
        end_train = time.time()
        
        # Bắt đầu tính thời gian kiểm thử
        start_test = time.time()
        y_pred = model.predict(X_test)
        end_test = time.time()
        
        # Tính thời gian và lưu lại
        train_time = end_train - start_train
        test_time = end_test - start_test
        week_results["train_times"].append(train_time)
        week_results["test_times"].append(test_time)

        # Đánh giá mô hình trên tập kiểm thử của fold hiện tại
        _, accuracy = model.evaluate(X_test, y_test, verbose=0)
        week_results["accuracy_per_fold"].append(accuracy)
        
        # Dự đoán
        y_pred_classes = y_pred.argmax(axis=1)
        y_test_classes = y_test.argmax(axis=1)
        
        # Tính các chỉ số cho mỗi fold
        precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='weighted')
        conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
        
        # Tính AUC-ROC
        try:
            # Tính AUC macro và theo từng lớp với OvR
            auc_macro = roc_auc_score(y_test, y_pred, multi_class="ovr", average="macro")
            auc_per_class = roc_auc_score(y_test, y_pred, multi_class="ovr", average=None)
            # Tính AUC weighted: tính trọng số theo số mẫu của từng lớp
            supports = np.bincount(y_test_classes, minlength=5)
            auc_weighted = np.sum(auc_per_class * supports) / np.sum(supports)
        except Exception as e:
            print(f"Lỗi khi tính AUC: {e}")
            auc_macro = np.nan
            auc_per_class = [np.nan] * 5
            auc_weighted = np.nan
            
        # Lưu kết quả của fold hiện tại
        week_results["precision_per_label"].append(precision)
        week_results["recall_per_label"].append(recall)
        week_results["f1_score_per_label"].append(f1)
        week_results["auc_roc_per_label"].append(auc_per_class)  # AUC từng lớp
        week_results["auc_roc_macro"].append(auc_macro)          # AUC macro
        week_results["auc_roc_weighted"].append(auc_weighted)      # AUC weighted
        week_results["confusion_matrices"].append(conf_matrix)
        week_results["precision_macro"].append(precision_macro)
        week_results["recall_macro"].append(recall_macro)
        week_results["f1_macro"].append(f1_macro)
        week_results["precision_weighted"].append(precision_weighted)
        week_results["recall_weighted"].append(recall_weighted)
        week_results["f1_weighted"].append(f1_weighted)

    # Tính trung bình cho từng nhãn
    average_precision_per_label = np.mean(week_results["precision_per_label"], axis=0)
    average_recall_per_label = np.nanmean(week_results["recall_per_label"], axis=0)
    average_f1_per_label = np.nanmean(week_results["f1_score_per_label"], axis=0)
    average_auc_per_label = np.nanmean(week_results["auc_roc_per_label"], axis=0)
    average_confusion_matrix = np.nanmean(week_results["confusion_matrices"], axis=0)
    average_train_time = sum(week_results["train_times"]) / len(week_results["train_times"])
    average_test_time = sum(week_results["test_times"]) / len(week_results["test_times"])
    average_accuracy = np.nanmean(week_results["accuracy_per_fold"])
    average_precision_macro = np.nanmean(week_results["precision_macro"])
    average_recall_macro = np.nanmean(week_results["recall_macro"])
    average_f1_macro = np.nanmean(week_results["f1_macro"])
    average_auc_macro = np.nanmean(week_results["auc_roc_macro"])
    average_precision_weighted = np.nanmean(week_results["precision_weighted"])
    average_recall_weighted = np.nanmean(week_results["recall_weighted"])
    average_f1_weighted = np.nanmean(week_results["f1_weighted"])
    average_auc_weighted = np.nanmean(week_results["auc_roc_weighted"])


    # Tạo DataFrame cho precision, recall, f1-score
    labels = np.unique(y_test_classes)  # Lấy nhãn từ y_test_classes
    metrics_df = pd.DataFrame({
        "Label": labels,
        "Average Precision": average_precision_per_label,
        "Average Recall": average_recall_per_label,
        "Average F1-Score": average_f1_per_label,
        "Average AUC": average_auc_per_label
    })
    
    # Tạo DataFrame cho confusion matrix
    confusion_df = pd.DataFrame(average_confusion_matrix, index=labels, columns=labels)
    # In kết quả Accuracy và Macro metrics
    print("\n=== Average Accuracy ===")
    print(f"{average_accuracy:.4f}")
    print("\n=== Average Macro Metrics ===")
    print(f"Macro Precision: {average_precision_macro:.4f}")
    print(f"Macro Recall: {average_recall_macro:.4f}")
    print(f"Macro F1-Score: {average_f1_macro:.4f}")
    print(f"Macro AUC-ROC: {average_auc_macro:.4f}")
    print("\n=== Average Weighted Metrics ===")
    print(f"Weighted Precision: {average_precision_weighted:.4f}")
    print(f"Weighted Recall: {average_recall_weighted:.4f}")
    print(f"Weighted F1-Score: {average_f1_weighted:.4f}")
    print(f"Weighted AUC-ROC: {average_auc_weighted:.4f}")
    print("\n=== Average Metrics per Label ===")
    print(metrics_df)
    print("\n=== Average Confusion Matrix ===")
    print(confusion_df)
    
    # Cập nhật kết quả cho tuần hiện tại
    week_results.update({
        "average_accuracy": average_accuracy,
        "average_precision_macro": average_precision_macro,
        "average_recall_macro": average_recall_macro,
        "average_f1_macro": average_f1_macro,
        "average_auc_macro": average_auc_macro,
        "average_precision_weighted": average_precision_weighted,
        "average_recall_weighted": average_recall_weighted,
        "average_f1_weighted": average_f1_weighted,
        "average_auc_weighted": average_auc_weighted,
        "average_metrics_df": metrics_df,
        "average_confusion_matrix": confusion_df,
        "average_train_times": average_train_time,
        "average_test_times": average_test_time,
    })
    overall_results_5folds.append(week_results)


Processing week1 with best parameters...
best parameters for week1: {'units_1': 192, 'dropout_1': 0.2, 'units_2': 96, 'dropout_2': 0.1, 'units_3': 64, 'dropout_3': 0.4, 'units_4': 192, 'dropout_4': 0.5, 'learning_rate': 0.002143981088964242}
Fold 1: Using file /kaggle/input/FillZero_minmax_baseline/clean_week1/train/5-folds/data_part_1.csv as test set
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


328/328 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.6129 - loss: 0.1498 - val_accuracy: 0.6744 - val_loss: 0.1186
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6574 - loss: 0.1224 - val_accuracy: 0.6748 - val_loss: 0.1148
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6627 - loss: 0.1199 - val_accuracy: 0.6668 - val_loss: 0.1163
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6745 - loss: 0.1171 - val_accuracy: 0.7053 - val_loss: 0.1093
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6815 - loss: 0.1150 - val_accuracy: 0.7160 - val_loss: 0.1082
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6739 - loss: 0.1197 - val_accuracy: 0.7091 - val_loss: 0.1053
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6722 - loss: 0.1165 - val_accuracy: 0.7133 - val_loss: 0.1076
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6813 - loss: 0.1150 - val_accuracy: 0.7125 - val_

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


328/328 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.6083 - loss: 0.1522 - val_accuracy: 0.6904 - val_loss: 0.1165
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6615 - loss: 0.1222 - val_accuracy: 0.7011 - val_loss: 0.1124
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6606 - loss: 0.1206 - val_accuracy: 0.6336 - val_loss: 0.1136
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6565 - loss: 0.1218 - val_accuracy: 0.7045 - val_loss: 0.1108
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6814 - loss: 0.1173 - val_accuracy: 0.7099 - val_loss: 0.1114
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6696 - loss: 0.1167 - val_accuracy: 0.7064 - val_loss: 0.1090
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6797 - loss: 0.1126 - val_accuracy: 0.7106 - val_loss: 0.1082
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6707 - loss: 0.1134 - val_accuracy: 0.7118 - val

...

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.6052 - loss: 0.1983 - val_accuracy: 0.6114 - val_loss: 0.1340
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6108 - loss: 0.1380 - val_accuracy: 0.6137 - val_loss: 0.1290
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6175 - loss: 0.1317 - val_accuracy: 0.5980 - val_loss: 0.1237
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6443 - loss: 0.1256 - val_accuracy: 0.6972 - val_loss: 0.1189
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6795 - loss: 0.1227 - val_accuracy: 0.6827 - val_loss: 0.1188
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6806 - loss: 0.1220 - val_accuracy: 0.7094 - val_loss: 0.1117
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6933 - loss: 0.1170 - val_accuracy: 0.7052 - val_loss: 0.1083
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6945 - loss: 0.1126 - val_accuracy: 0.

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


328/328 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.5925 - loss: 0.1990 - val_accuracy: 0.6114 - val_loss: 0.1337
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6201 - loss: 0.1330 - val_accuracy: 0.6114 - val_loss: 0.1290
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6189 - loss: 0.1286 - val_accuracy: 0.5988 - val_loss: 0.1282
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.6041 - loss: 0.1300 - val_accuracy: 0.5973 - val_loss: 0.1224
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6372 - loss: 0.1240 - val_accuracy: 0.6472 - val_loss: 0.1206
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6670 - loss: 0.1224 - val_accuracy: 0.6957 - val_loss: 0.1164
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6910 - loss: 0.1197 - val_accuracy: 0.6976 - val_loss: 0.1129
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6995 - loss: 0.1156 - val_accuracy: 0.6968 - val_

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
# Duyệt qua các tuần trong overall_results
for week_result in overall_results_5folds:
    week = week_result["week"]
    average_train_time = np.mean(week_result["train_times"])
    average_test_time = np.mean(week_result["test_times"])
    average_metrics_df = week_result["average_metrics_df"]
    average_accuracy = np.mean(week_results["accuracy_per_fold"])
    average_confusion_matrix = week_result["average_confusion_matrix"]
    
    # In kết quả
    print(f"\n=== Results for {week} ===")
    print(f"Average Accurancy: {average_accuracy}")
    print(f"Average Train Time: {average_train_time:.4f} seconds")
    print(f"Average Test Time: {average_test_time:.4f} seconds")
    print(f"Average AUC Macro: {average_auc_macro}")
    print(f"Average AUC Weighted: {average_auc_weighted}")
    print("\nAverage Precision, Recall, F1-Score, AUC-ROC per Label:")
    print(average_metrics_df)
    print("\nAverage Confusion Matrix:")
    print(average_confusion_matrix)



=== Results for week1 ===
Average Accurancy: 0.7755487322807312
Average Train Time: 131.8849 seconds
Average Test Time: 0.7430 seconds
Average AUC Macro: 0.8755450314290574
Average AUC Weighted: 0.8957033596522515

Average Precision, Recall, F1-Score, AUC-ROC per Label:
   Label  Average Precision  Average Recall  Average F1-Score  Average AUC
0      0           0.599238        0.639000          0.617141     0.824654
1      1           0.166667        0.022727          0.036632     0.865387
2      2           0.657650        0.469690          0.540162     0.842286
3      3           0.567452        0.275449          0.324648     0.854135
4      4           0.796112        0.873392          0.832418     0.850459

Average Confusion Matrix:
       0    1     2     3       4
0  383.4  2.2  12.4  11.2   190.8
1   52.0  2.0   1.2   2.2    30.2
2   38.2  0.8  77.2   3.6    44.6
3   24.2  0.0   2.4  46.0    94.4
4  145.0  2.0  27.2  28.8  1400.4

=== Results for week2 ===
Average Accurancy: 0

## Kiểm tra trên tập test

In [11]:
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras import backend as K

# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    
    return focal_loss_fixed

def build_lstm_model(params):
    model = tf.keras.Sequential()
    
    # LSTM layer 1
    model.add(layers.LSTM(
        units=params.get('units_1', 32),
        return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]))
    )
    model.add(layers.Dropout(rate=params.get('dropout_1', 0.2)))

    # LSTM layer 2
    model.add(layers.LSTM(
        units=params.get('units_2', 32),
        return_sequences=True
    ))
    model.add(layers.Dropout(rate=params.get('dropout_2', 0.2)))

    # LSTM layer 3
    model.add(layers.LSTM(
        units=params.get('units_3', 32),
        return_sequences=True
    ))
    model.add(layers.Dropout(rate=params.get('dropout_3', 0.2)))

    # LSTM layer 4
    model.add(layers.LSTM(
        units=params.get('units_4', 32),
        return_sequences=False
    ))
    model.add(layers.Dropout(rate=params.get('dropout_4', 0.2)))

    # Output layer
    model.add(layers.Dense(5, activation='softmax'))  # 5 classes in output

    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=params['learning_rate']),
        loss=focal_loss(gamma=params.get('gamma', 2.), alpha=params.get('alpha', 0.25)),
        metrics=['accuracy']
    )
    
    return model

In [12]:
# Mảng lưu dữ liệu của các tuần
results = []

def process_week(week_num, best_params, results):
    print(f"\n=== Processing Week {week_num} ===")
    params = best_params[f"week{week_num}"].values
    # Đường dẫn tới dữ liệu tuần tương ứng
    train_path = f"{BASE_PATH}/clean_week{week_num}/train/clean_data_week{week_num}.csv"
    test_path = f"{BASE_PATH}/clean_week{week_num}/test/test_week{week_num}.csv"
    
    # Load dữ liệu
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)
    
    # Tách X và y
    X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                       "course_id", "school", "enroll_time", "classification"])
    y_train = train_data['classification_encoded']
    
    X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                     "course_id", "school", "enroll_time", "classification"])
    y_test = test_data['classification_encoded']

    # Áp dụng SMOTE cho tập huấn luyện
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Chuyển đổi nhãn sang dạng one-hot
    y_train_resampled = to_categorical(y_train_resampled, num_classes=5)
    y_test = to_categorical(y_test, num_classes=5)

    # Reshape dữ liệu cho LSTM
    X_train_resampled = X_train_resampled.to_numpy().reshape((X_train_resampled.shape[0], 1, X_train_resampled.shape[1]))
    X_test = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))

    # Xây dựng mô hình với tham số tốt nhất
    model = build_lstm_model(params)
    
    # Huấn luyện mô hình
    start_train = time.time()
    model.fit(X_train_resampled, y_train_resampled, epochs=50, validation_split=0.1, batch_size=32)
    end_train = time.time()
    
    # Kiểm thử mô hình
    start_test = time.time()
    y_pred = model.predict(X_test)
    end_test = time.time()
    
    # Tính thời gian huấn luyện và kiểm thử
    train_time = end_train - start_train
    test_time = end_test - start_test
    
    # Đánh giá mô hình
    y_pred_classes = y_pred.argmax(axis=1)
    y_test_classes = y_test.argmax(axis=1)
    
    precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='macro')
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='weighted')
    conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
    accuracy = accuracy_score(y_test_classes, y_pred_classes)
    
    # Tính AUC-ROC (với one-vs-rest)
    try:
        auc_macro = roc_auc_score(y_test, y_pred, multi_class="ovr", average="macro")
        auc_per_class = roc_auc_score(y_test, y_pred, multi_class="ovr", average=None)
        # Tính AUC weighted tự tính theo trọng số mẫu của từng lớp
        supports = np.bincount(y_test_classes, minlength=5)
        auc_weighted = np.sum(auc_per_class * supports) / np.sum(supports)
    except Exception as e:
        print(f"Lỗi khi tính AUC: {e}")
        auc_macro = np.nan
        auc_per_class = [np.nan] * 5
        auc_weighted = np.nan

    # Lưu kết quả vào mảng
    results.append({
        "week": week_num,
        "train_time": train_time,
        "test_time": test_time,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "accuracy": accuracy,
        "precision_macro": precision_macro,
        "recall_macro": recall_macro,
        "f1_macro": f1_macro,
        "precision_weighted": precision_weighted,
        "recall_weighted": recall_weighted,
        "f1_weighted": f1_weighted,
        "auc_macro": auc_macro,
        "auc_weighted": auc_weighted,
        "auc_per_class": auc_per_class,
        "confusion_matrix": conf_matrix
    })
    
    # In kết quả chi tiết
    print("\n=== Precision, Recall, F1-Score per Label ===")
    print(pd.DataFrame({
        "Label": np.unique(y_test_classes),
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1
    }))

    print("\n=== Macro Averages & Accuracy ===")
    print(f"Macro Precision: {precision_macro:.4f}")
    print(f"Macro Recall: {recall_macro:.4f}")
    print(f"Macro F1-Score: {f1_macro:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    
    print("\n=== Weighted Averages ===")
    print(f"Weighted Precision: {precision_weighted:.4f}")
    print(f"Weighted Recall: {recall_weighted:.4f}")
    print(f"Weighted F1-Score: {f1_weighted:.4f}")
    
    print("\n=== AUC-ROC ===")
    print(f"AUC Macro: {auc_macro:.4f}")
    print(f"AUC Weighted: {auc_weighted:.4f}")
    print(f"AUC per Label: {auc_per_class}")
    
    print("\n=== Confusion Matrix ===")
    print(pd.DataFrame(conf_matrix, index=np.unique(y_test_classes), columns=np.unique(y_test_classes)))
    
    print(f"\nTrain Time: {train_time:.2f} seconds")
    print(f"Test Time: {test_time:.2f} seconds")

In [13]:
process_week(1, best_params, results)


=== Processing Week 1 ===
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1128/1128 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.4516 - loss: 0.2025 - val_accuracy: 0.6139 - val_loss: 0.2499
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.5156 - loss: 0.1824 - val_accuracy: 0.6169 - val_loss: 0.2648
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.5336 - loss: 0.1762 - val_accuracy: 0.5929 - val_loss: 0.2424
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.5491 - loss: 0.1732 - val_accuracy: 0.6697 - val_loss: 0.2407
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.5733 - loss: 0.1654 - val_accuracy: 0.7316 - val_loss: 0.2361
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.5741 - loss: 0.1663 - val_accuracy: 0.6852 - val_loss: 0.2337
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.5736 - loss: 0.1653 - val_accuracy: 0.7104 - val_loss: 0.2131
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.5869 - loss: 0.1617 - val_accura

In [14]:
process_week(2, best_params, results)


=== Processing Week 2 ===
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1128/1128 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - accuracy: 0.4446 - loss: 0.2005 - val_accuracy: 0.6169 - val_loss: 0.2324
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.5181 - loss: 0.1836 - val_accuracy: 0.7194 - val_loss: 0.2389
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.5397 - loss: 0.1748 - val_accuracy: 0.6855 - val_loss: 0.2415
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.5567 - loss: 0.1709 - val_accuracy: 0.6875 - val_loss: 0.2296
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.5675 - loss: 0.1681 - val_accuracy: 0.5799 - val_loss: 0.2530
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.5786 - loss: 0.1638 - val_accuracy: 0.6972 - val_loss: 0.2327
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.5908 - loss: 0.1590 - val_accuracy: 0.6615 - val_loss: 0.2277
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.5969 - loss: 0.1567 - val_accura

In [15]:
process_week(3, best_params, results)


=== Processing Week 3 ===
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1128/1128 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - accuracy: 0.4172 - loss: 0.2114 - val_accuracy: 0.3577 - val_loss: 0.2750
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.4880 - loss: 0.1911 - val_accuracy: 0.7141 - val_loss: 0.2414
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.5223 - loss: 0.1815 - val_accuracy: 0.6468 - val_loss: 0.2362
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.5156 - loss: 0.1816 - val_accuracy: 0.6371 - val_loss: 0.2193
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.5236 - loss: 0.1791 - val_accuracy: 0.6278 - val_loss: 0.2378
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.5329 - loss: 0.1737 - val_accuracy: 0.6114 - val_loss: 0.2625
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.5446 - loss: 0.1713 - val_accuracy: 0.7082 - val_loss: 0.2226
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.5441 - loss: 0.1720 - val_accura

In [16]:
process_week(4, best_params, results)


=== Processing Week 4 ===
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1128/1128 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.3489 - loss: 0.2278 - val_accuracy: 0.4707 - val_loss: 0.2596
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.5429 - loss: 0.1795 - val_accuracy: 0.6593 - val_loss: 0.2472
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.5761 - loss: 0.1683 - val_accuracy: 0.6909 - val_loss: 0.2189
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.5977 - loss: 0.1613 - val_accuracy: 0.6939 - val_loss: 0.2235
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.6120 - loss: 0.1575 - val_accuracy: 0.6782 - val_loss: 0.2268
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.6102 - loss: 0.1562 - val_accuracy: 0.6807 - val_loss: 0.2196
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.6199 - loss: 0.1510 - val_accuracy: 0.7057 - val_loss: 0.2053
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.6198 - loss: 0.1491 - val_accura

In [17]:
# Hiển thị dữ liệu của các tuần
print("\n=== Summary Results for All Weeks ===")
for result in results:
    print(f"Week {result['week']}:")
    print(f"  Train Time: {result['train_time']:.2f} seconds")
    print(f"  Test Time: {result['test_time']:.2f} seconds")
    print(f"  Accurancy: {result['accuracy']}")
    print(f"  Precision: {result['precision']}")
    print(f"  Recall: {result['recall']}")
    print(f"  F1-Score: {result['f1_score']}")
    print(f"  Macro Precision: {result['precision_macro']}")
    print(f"  Macro Recall: {result['recall_macro']}")
    print(f"  Macro F1-Score: {result['f1_macro']}")
    print(f"  Confusion Matrix:\n{result['confusion_matrix']}")
    print("\n=== AUC-ROC ===")
    print(f"AUC Macro: {result['auc_macro']:.4f}")
    print(f"AUC Weighted: {result['auc_weighted']:.4f}")
    print(f"AUC per Label: {result['auc_per_class']}")


=== Summary Results for All Weeks ===
Week 1:
  Train Time: 431.22 seconds
  Test Time: 0.70 seconds
  Accurancy: 0.7054878048780487
  Precision: [0.70226537 0.17427386 0.54545455 0.51219512 0.89976959]
  Recall: [0.57866667 0.77777778 0.52427184 0.6        0.77866401]
  F1-Score: [0.63450292 0.28474576 0.53465347 0.55263158 0.83484768]
  Macro Precision: 0.5667916967497335
  Macro Recall: 0.6518760594161421
  Macro F1-Score: 0.5682762812044923
  Confusion Matrix:
[[217  84  14   2  58]
 [  3  42   2   0   7]
 [ 16  23  54   2   8]
 [ 10  13   5  63  14]
 [ 63  79  24  56 781]]

=== AUC-ROC ===
AUC Macro: 0.8738
AUC Weighted: 0.8697
AUC per Label: [0.85332385 0.87600999 0.87350847 0.8932868  0.87264502]
Week 2:
  Train Time: 427.83 seconds
  Test Time: 0.66 seconds
  Accurancy: 0.7365853658536585
  Precision: [0.72477064 0.24550898 0.47619048 0.48360656 0.90311804]
  Recall: [0.632      0.75925926 0.58252427 0.56190476 0.80857428]
  F1-Score: [0.67521368 0.37104072 0.52401747 0.519823